In [1]:
import numpy as np
import matplotlib.pyplot as plt
from lib import sig_gen
import torch
import torch.nn as nn
# import torch.optim
torch.__version__

'1.0.1.post2'

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
beta, span, sps = 0.4, 4, 6
rrc = sig_gen.rrcosdesign(beta, span, sps)
rrc = torch.from_numpy(rrc).float().to(device)

rrc

tensor([3.1963, 3.1626, 3.0999, 3.0261, 2.9711, 2.9694, 3.0484, 3.2176, 3.4615,
        3.7407, 4.0004, 4.1844, 4.2509, 4.1844, 4.0004, 3.7407, 3.4615, 3.2176,
        3.0484, 2.9694, 2.9711, 3.0261, 3.0999, 3.1626], device='cuda:0')

In [31]:
symb=64
M=4
msg = torch.empty(symb, dtype=torch.float, device=device).random_(0, M)
# sig_mod = (np.pi/M+msg*(2*np.pi/M))*1j
sig_mod_r = (np.pi/M+msg*(2*np.pi/M)).cos()
sig_mod_i = (np.pi/M+msg*(2*np.pi/M)).sin()

sig_mod_r

tensor([ 0.7071, -0.7071,  0.7071, -0.7071,  0.7071, -0.7071, -0.7071, -0.7071,
         0.7071,  0.7071, -0.7071, -0.7071,  0.7071,  0.7071,  0.7071, -0.7071,
         0.7071, -0.7071, -0.7071, -0.7071,  0.7071, -0.7071,  0.7071, -0.7071,
        -0.7071,  0.7071, -0.7071, -0.7071, -0.7071, -0.7071, -0.7071,  0.7071,
         0.7071,  0.7071, -0.7071, -0.7071,  0.7071, -0.7071, -0.7071, -0.7071,
         0.7071,  0.7071,  0.7071, -0.7071,  0.7071,  0.7071, -0.7071,  0.7071,
         0.7071,  0.7071, -0.7071,  0.7071, -0.7071, -0.7071, -0.7071,  0.7071,
         0.7071, -0.7071, -0.7071,  0.7071, -0.7071, -0.7071, -0.7071, -0.7071],
       device='cuda:0')

In [ ]:
def psk_gen(symb=64, M=4, beta=0.4, span=4, sps=16, rrc, device):
    """
    generate baseband PSK signal
    """
#     rrc = rrcosdesign(beta, span, sps)
    msg = torch.empty(symb, dtype=torch.float, device=device).random_(0, M) 
    sig_mod = torch.exp(1j*(np.pi/M+msg*(2*np.pi/M)))
    sig_up = upsample(sig_mod, sps)
    sig_pulse = np.convolve(sig_up, rrc)
    return sig_pulse[int(sps*span/2):int(1-sps*span/2)]

In [ ]:
sig2 = np.vstack((sig, sig, sig))
sig2 = np.expand_dims(sig2, 1)[:, :, 0:-1: 4]

In [ ]:
class CVcnn(torch.nn.Module):
    def __init__(self):
        super(CVcnn, self).__init__()
        
        self.conv1_r = nn.Conv1d(1, 8, 5)
        self.conv1_i = nn.Conv1d(1, 8, 5)
        
        self.conv2_r = nn.Conv1d(8, 8, 5)
        self.conv2_i = nn.Conv1d(8, 8, 5)
        
    def forward(self, x_r, x_i):
        (y_r, y_i) = (self.conv1_r(x_r) - self.conv1_i(x_i),
                      self.conv1_r(x_i) + self.conv1_i(x_r))
        
        (y_r, y_i) = (self.conv2_r(y_r) - self.conv2_i(y_i),
                      self.conv2_r(y_i) + self.conv2_i(y_r))
        
        return y_r, y_i
    
    
net = CVcnn()
print('Number of parameters: {}'.format(sum(p.numel() for p in net.parameters())))

x_r = torch.from_numpy(sig2.real).float()
x_i = torch.from_numpy(sig2.imag).float()
y_r, y_i = net(x_r, x_i)

print(x_r.shape, y_r.shape)

In [ ]:
class RVcnn(torch.nn.Module):
    def __init__(self):
        super(RVcnn, self).__init__()
        
        self.conv1 = nn.Conv1d(2, 8, 5)
        self.conv2 = nn.Conv1d(8, 8, 5)
#         self.conv1_i = nn.Conv1d(1, 2, 5) 
    def forward(self, x):
        y = self.conv1(x)
        y = self.conv2(y)
#         y_r = self.conv1_r(x_r) - self.conv1_i(x_i)
#         y_i = self.conv1_r(x_i) + self.conv1_i(x_r)
        return y

net = RVcnn()
print('Number of parameters: {}'.format(sum(p.numel() for p in net.parameters())))

x_r = torch.from_numpy(sig2.real).float()
x_i = torch.from_numpy(sig2.imag).float()
x = torch.cat((x_r, x_i), 1)
y= net(x)

print(x.shape, y.shape)